<a href="https://colab.research.google.com/github/sisifo3/P_T_3/blob/main/Densenet121_CheXpert_NIH_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive._mount('/content/drive')

Mounted at /content/drive


In [2]:
!cp '/content/drive/MyDrive/NIH_Dataset_total/just_test_cheXpert_NIH' -r '/content/'

In [21]:
import cv2
import numpy as np
from PIL import Image
from scipy.ndimage.filters import gaussian_filter
from scipy.ndimage.interpolation import map_coordinates

random_state = np.random.RandomState(0)

def clahe(image_name):
    image = cv2.imread(image_name, 0)
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
    he_img = clahe.apply(image)
    image = cv2.cvtColor(he_img, cv2.COLOR_GRAY2RGB)

    return Image.fromarray(image)


In [22]:
from torchvision import transforms
import torch

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


#for CheXpert
transform = transforms.Compose([
                                transforms.Resize((320, 320)),
                                transforms.ToTensor(),
                                transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
                                ])

In [29]:
import random
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from PIL import Image
import torch
from torch.utils.data import Dataset

def getImagesLabels(filename1,filename2):

    df1 = pd.read_csv(filename1)
    df2 = pd.read_csv(filename2)
    df = pd.concat([df1, df2]) 

    relevant_cols = ['path','No Finding', 'Enlarged Cardiomediastinum', 'Cardiomegaly',
                    'Lung Opacity', 'Lung Lesion', 'Edema',  'Consolidation',
                    'Pneumonia', 'Atelectasis','Pneumothorax', 'Pleural Effusion',
                    'Pleural Other', 'Fracture', 'Support Devices','Hernia','Mass',
                    'Fibrosis','Infiltration','Nodule','Emphysema','Pleural_Thickening']


    df = df[relevant_cols]
    df = df.replace(np.nan, 0.0)

    df = df.replace(-1.0, 0.0)


    X = df['path']
    y = df[['No Finding', 'Enlarged Cardiomediastinum', 'Cardiomegaly',
                    'Lung Opacity', 'Lung Lesion', 'Edema',  'Consolidation',
                    'Pneumonia', 'Atelectasis','Pneumothorax', 'Pleural Effusion',
                    'Pleural Other', 'Fracture', 'Support Devices','Hernia','Mass',
                    'Fibrosis','Infiltration','Nodule','Emphysema','Pleural_Thickening']]

    return np.asarray(X), np.asarray(y)



#filename = '/content/drive/MyDrive/NIH_Dataset_total/final_cheXpert_NIH_train.csv'
#x1, y1 = getImagesLabels(filename)

In [24]:
class CheXpertDataset(Dataset):
    def __init__(self, image_list, labels, transform=None, test=False):
        self.image_names = image_list
        self.labels = labels
        self.transform = transform
        self.test = test

    def __getitem__(self, index):
        image_name = self.image_names[index]
        image = clahe(image_name)
        label = self.labels[index]
        image = self.transform(image)
        return image, torch.FloatTensor(label)

    def __len__(self):
       
        return len(self.image_names)

In [25]:
import torch
import  torchvision
import torch.nn as nn
from torchsummary import summary

class DenseNet121(nn.Module):
    def __init__(self, out_size, test=False):
        super(DenseNet121, self).__init__()
        self.test = test
        self.densenet121 = torchvision.models.densenet121(pretrained=True)
        num_ftrs = self.densenet121.classifier.in_features
        self.densenet121.classifier = nn.Sequential(
            nn.Linear(num_ftrs, num_ftrs),
            nn.BatchNorm1d(num_ftrs),
            nn.ReLU(inplace=True),
            nn.Dropout(0.5),
            nn.Linear(num_ftrs, num_ftrs//2),
            nn.BatchNorm1d(num_ftrs//2),
            nn.ReLU(inplace=True),
            nn.Dropout(0.25),
            nn.Linear(num_ftrs//2, out_size),
        )

    def forward(self, x):
        x = self.densenet121(x)

        if self.test:
            #return torch.sigmoid(x)
            return x
        else:
            return x


In [30]:

import os

import matplotlib.pyplot as plt
import numpy as np
import sklearn.metrics as metrics
from sklearn.metrics import accuracy_score
#from sklearn.metrics.ranking import roc_auc_score
import torch
import torch.backends.cudnn as cudnn
from torch.utils.data import DataLoader


def test(test_loader, model,  device, num_classes, class_names):
        cudnn.benchmark = True
        threshold = 0.45

        #if checkpoint is not None:
            #checkpoint = torch.load(checkpoint,map_location ='cpu')
            #model.load_state_dict(checkpoint['model_state_dict'])

        gt = torch.FloatTensor().to(device)
        pred = torch.FloatTensor().to(device)


        model.eval()
        with torch.no_grad():
            for i, (data, target) in enumerate(test_loader):
                target = target.type(torch.int64)
                target = torch.zeros(target.size(0), 14).scatter_(1, target, 1.)
                
                target = target.to(device)
                gt = torch.cat((gt, target), 0).to(device)

                out = model(data)
                print("output logits")
                #print(class_names)
                print(out)
                for i in range(num_classes):
                  print(out[0,i], class_names[i])
                out = torch.sigmoid(out)
                print("\n output sigmoid prediction")
                print(out)
                for i in range(num_classes):
                  print(out[0,i], class_names[i])
                pred = torch.cat((pred, out), 0)
                pred[pred>=0.75] = 1
                pred[pred<0.75] = 0
                print("output label prediction")
                print(pred)
                for i in range(num_classes):
                  print(pred[0,i], class_names[i])



        return pred, gt


num_classes = 21
class_names = ['No Finding', 'Enlarged Cardiomediastinum', 'Cardiomegaly',
                    'Lung Opacity', 'Lung Lesion', 'Edema',  'Consolidation',
                    'Pneumonia', 'Atelectasis','Pneumothorax', 'Pleural Effusion',
                    'Pleural Other', 'Fracture', 'Support Devices','Hernia','Mass',
                    'Fibrosis','Infiltration','Nodule','Emphysema','Pleural_Thickening']


filename1 = '/content/just_test_cheXpert_NIH/final_Only_cheXpert_test.csv'
filename2 = '/content/just_test_cheXpert_NIH/final_Only_NIH_test_v2.csv'

test_images, test_labels = getImagesLabels(filename1, filename2)


#print('Testing on {} images\n'.format(len(test_images)))

test_dataset = CheXpertDataset(test_images[:4], test_labels[:4],transform, test=True)
test_loader = DataLoader(test_dataset, batch_size=4, shuffle=False,  num_workers=2, pin_memory=True)

model = DenseNet121(num_classes, test=True)
model = torch.nn.DataParallel(model)

chckpoints = '/content/drive/MyDrive/NIH_Dataset_total/weights_densenet121_CheXpert_NIH/weights_Densenet121_v10.pth'
model.load_state_dict(torch.load(chckpoints,map_location ='cpu'))

#checkpoint = '/content/drive/MyDrive/weightsDensenet121_uignoring/27-10-2021-21:44:49_model.pt'

pred, gt = test(test_loader, model, device, num_classes, class_names) 



output logits
tensor([[ 1.2431, -4.5262, -1.0284, -4.2945, -4.8121, -2.0949, -0.4309, -0.4359,
         -0.4297, -0.4291, -1.1597, -4.8088, -4.7880, -4.3705, -1.0212,  0.4997,
          0.4594,  0.7465,  0.6482,  0.1817,  0.3603],
        [ 1.2579, -4.5504, -1.0233, -4.3624, -4.8641, -2.1278, -0.4010, -0.3838,
         -0.4194, -0.4384, -1.1748, -4.8536, -4.8536, -4.4538, -1.3226,  0.3391,
          0.2068,  0.6020,  0.5015,  0.0087,  0.1911],
        [ 1.2409, -4.5258, -1.0256, -4.3054, -4.8195, -2.0964, -0.4203, -0.4221,
         -0.4263, -0.4274, -1.1606, -4.8181, -4.7990, -4.3834, -1.0983,  0.4646,
          0.3989,  0.7138,  0.6145,  0.1441,  0.3233],
        [ 1.2531, -4.6014, -1.0382, -4.3736, -4.9024, -2.1191, -0.4262, -0.4283,
         -0.4325, -0.4361, -1.1731, -4.9034, -4.8819, -4.4519, -1.1248,  0.4681,
          0.3935,  0.7230,  0.6218,  0.1427,  0.3250]])
tensor(1.2431) No Finding
tensor(-4.5262) Enlarged Cardiomediastinum
tensor(-1.0284) Cardiomegaly
tensor(-4.2945) Lun